In [ ]:
# imports
import random
import copy
import sys
sys.path.append("../")

from scripts.load_data import write_tsv_file, extract_labeled_tokens, mapping, read_tsv_file, write_iob2_file
from middle_eastern_ne import extract_first_names, get_last_names,  load_location, load_organisation

random.seed(42)